- iterate over the cells
- save hydrology and sediments
- make sure to save the ls, sc

1. Experiment 1: NO GLACIERS
2. Experiment 2: transport limited case (!!! HERE: calculate what is the mean annual sedimant yeild) 
3. Experiment 3: linear increase in sediment storage: (HOW BIG LANDSLIDE SHOULD BE?)

-- to make changes in the model go to run.py, SedCas_glacier_sed.py (model) or modules.py (modules)

In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt


# modelling part
from run import run


# transport limited case (=sediment unlimited): calculate how much sediment there MAY be

In [ ]:

import os
from run import run

climatepath = '/Users/varyabazilova/Desktop/paper2/downscaling_simple/transport_limited/langtang'

# get all folders togetehr 
# ls_ = os.listdir('./langtang')
ls_ = os.listdir(climatepath)
ls = [d for d in ls_ if not d.startswith('.')]
dir0 = os.getcwd()

# run the model for each
for d in ls:
    path = os.path.join(dir0, 'langtang', d)
    print(path)
    os.chdir(path)
    run()
    os.chdir(dir0)


# how much sediment is exported per year in the transport limited case (= unlimited amount of sedoments) ?

In [ ]:
# for each output sediment folder calculate the total annual 
# go over each folder 
# take Sediment out file 
# calculate annual mean
# save into the table, where the table has: cellnr, mean annual sed yeald

In [4]:

# Define your custom function
def annual_sum_mean(df):
    df['D'] = pd.to_datetime(df['D'])
    df['Year'] = df['D'].dt.year
    annual_sum = df.groupby('Year').sum()
    annual_mean_Qstl = annual_sum.Qstl.mean()
    return annual_mean_Qstl

def annual_sum_percentile(df, percentile):
    df['D'] = pd.to_datetime(df['D'])
    df['Year'] = df['D'].dt.year
    annual_sum = df.groupby('Year').sum()
    annual_percentile_Qstl = annual_sum.Qstl.quantile(percentile / 100)
    return annual_percentile_Qstl

# def annual_sum_25percent(df):
#     df['D'] = pd.to_datetime(df['D'])
#     df['Year'] = df['D'].dt.year
#     annual_sum = df.groupby('Year').sum()
#     annual_25_Qstl = annual_sum.Qstl.quantile(0.25)
#     return annual_25_Qstl

# def annual_sum_50percent(df):
#     df['D'] = pd.to_datetime(df['D'])
#     df['Year'] = df['D'].dt.year
#     annual_sum = df.groupby('Year').sum()
#     annual_50_Qstl = annual_sum.Qstl.quantile(0.5)
#     return annual_50_Qstl

# def annual_sum_75percent(df):
#     df['D'] = pd.to_datetime(df['D'])
#     df['Year'] = df['D'].dt.year
#     annual_sum = df.groupby('Year').sum()
#     annual_50_Qstl = annual_sum.Qstl.quantile(0.5)
#     return annual_50_Qstl


In [34]:

# Path to the directory containing your folders
directory_path = '/Users/varyabazilova/Desktop/paper2/downscaling_simple/transport_limited/langtang'

# an empty table to store results
mean_annual_sum = []
mean_annual_25 = []
mean_annual_50 = []
mean_annual_75 = []

# Iterate over each folder
for folder_name in os.listdir(directory_path):
    folder_path = os.path.join(directory_path, folder_name)
    
    # Check if it's a directory and its name starts with 'cellnr'
    if os.path.isdir(folder_path) and folder_name.startswith('cellnr'):
        # Locate the 'sediment.out' file within the folder
        sediment_out_path = os.path.join(folder_path, 'Sediment.out')
        
        # Read the contents of the file into a pandas DataFrame
        df = pd.read_csv(sediment_out_path, delimiter=',')  # Adjust delimiter if needed
        # mean:
        mean_annual_sum_value = annual_sum_mean(df)
        # percentiles:
        mean_annual_25_value = annual_sum_percentile(df, 25)
        mean_annual_50_value = annual_sum_percentile(df, 50)
        mean_annual_75_value = annual_sum_percentile(df, 75)

        
        # Save the DataFrame into the results table
        mean_annual_sum.append((int(folder_name.replace('cellnr', '')), mean_annual_sum_value))
        mean_annual_25.append((int(folder_name.replace('cellnr', '')), mean_annual_25_value))
        mean_annual_50.append((int(folder_name.replace('cellnr', '')), mean_annual_50_value))
        mean_annual_75.append((int(folder_name.replace('cellnr', '')), mean_annual_75_value))

# make table
mean_annual_sum_df = pd.DataFrame(mean_annual_sum, columns=['cellnr', 'annual_mean_Qstl_mm']).set_index('cellnr')
mean_annual_25_df = pd.DataFrame(mean_annual_25, columns=['cellnr', 'annual_25percent_Qstl_mm']).set_index('cellnr')#, inplace=True)
mean_annual_50_df = pd.DataFrame(mean_annual_50, columns=['cellnr', 'annual_50percent_Qstl_mm']).set_index('cellnr')#, inplace=True)
mean_annual_75_df = pd.DataFrame(mean_annual_75, columns=['cellnr', 'annual_75percent_Qstl_mm']).set_index('cellnr')#, inplace=True)

# merge together
merged_df = pd.concat([mean_annual_sum_df, mean_annual_25_df, mean_annual_50_df, mean_annual_75_df], axis=1)


## convert to m3


In [35]:
for column in merged_df.columns:
    # Check if the column name contains '_mm'
    if '_mm' in column:
        # Replace '_mm' with '_m3' and create a new column
        new_column_name = column.replace('_mm', '_m3')
        merged_df[new_column_name] = merged_df[column] * 4.83 * 10. ** -3

## m3/day

In [47]:
# Iterate over each column in the DataFrame
for column in merged_df.columns:
    if '_m3' in column:
        merged_df[column + '_day'] = merged_df[column] /365


# add coordinates and save the output 

In [51]:
# csv with coordinates and geopotential (z) 
langtang_elevation = pd.read_csv('/Users/varyabazilova/Desktop/paper2/Thesisfiles_Varya/climate/elevation/geopotential_langtang.csv').set_index('cellnr')
with_coords = pd.concat([merged_df, langtang_elevation],  axis=1).reset_index()




In [53]:
#save alltogether 
output_path = '/Users/varyabazilova/Desktop/paper2/downscaling_simple/transport_limited/langtang/'
# with_coords.to_csv(output_path + 'langtang_tl_percentiles_with_coors.csv', index = False)

In [54]:
with_coords.columns

Index(['cellnr', 'annual_mean_Qstl_mm', 'annual_25percent_Qstl_mm',
       'annual_50percent_Qstl_mm', 'annual_75percent_Qstl_mm',
       'annual_mean_Qstl_m3', 'annual_25percent_Qstl_m3',
       'annual_50percent_Qstl_m3', 'annual_75percent_Qstl_m3',
       'annual_mean_Qstl_m3_day', 'annual_25percent_Qstl_m3_day',
       'annual_50percent_Qstl_m3_day', 'annual_75percent_Qstl_m3_day',
       'Latitude', 'Longitude', 'z'],
      dtype='object')

In [63]:
columns_to_copy = ['cellnr', 'annual_mean_Qstl_m3_day', 'annual_25percent_Qstl_m3_day',
                   'annual_50percent_Qstl_m3_day', 'annual_75percent_Qstl_m3_day',
                   'Latitude', 'Longitude']

selected_columns= with_coords[columns_to_copy].copy()
                              

In [64]:

# selected_columns.to_csv(output_path + 'langtang_tl_percentiles_with_coors_m3_day.csv', index = False)

# test for 1 file

In [ ]:
testsed = pd.read_csv('/Users/varyabazilova/Desktop/paper2/downscaling_simple/transport_limited/langtang/cellnr28/Sediment.out')
testsed

df = annual_sum_percentile(testsed, 25)





In [ ]:

df = testsed

df['D'] = pd.to_datetime(df['D'])
df['Year'] = df['D'].dt.year
annual_sum = df.groupby('Year').sum()


In [ ]:
plt.figure(figsize=(20,7))
plt.bar(height = annual_sum.Qstl, x = annual_sum.index, label = 'Qstl')
plt.bar(height = annual_sum.Qdftl, x = annual_sum.index, color = 'lime', alpha = 0.5, label = 'Qdstl')

# plt.bar(height = annual_sum.Q100, x = annual_sum.index, color = 'magenta', alpha = 0.5, label = 'Q100')
plt.legend()

In [ ]:
plt.figure(figsize=(20,7))
# plt.bar(height = annual_sum.Qstl, x = annual_sum.index, label = 'Qstl')
# plt.bar(height = annual_sum.Qdftl, x = annual_sum.index, color = 'lime', alpha = 0.5, label = 'Qdstl')

plt.bar(height = annual_sum.Q100, x = annual_sum.index, color = 'magenta', alpha = 0.5, label = 'Q100')
plt.legend()

In [ ]:
annual_sum.describe()

In [ ]:
# test climate input
import pandas as pd
testclimate = pd.read_csv('/Users/varyabazilova/Desktop/paper2/downscaling_simple/langtang/cellnr12/climatecell.12.met')

In [ ]:
testsed = pd.read_csv('/Users/varyabazilova/Desktop/paper2/downscaling_simple/langtang/cellnr28/Sediment.out')
testsed

df = testsed


df['D'] = pd.to_datetime(df['D'])
df['Year'] = df['D'].dt.year
annual_sum = df.groupby('Year').sum()
annual_mean_Qstl = annual_sum.Qstl.mean()

In [ ]:
annual_sum.Qstl.mean()

## 